# IMPORTS

In [2]:
# 0. Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid")  
pd.options.display.max_columns = 200

In [3]:
# 1. Load cleaned data
df = pd.read_csv('Aircraft_Fault_Dataset_cleaned.csv', parse_dates=['Date'])
print("Rows:", len(df))
df.head()
df.info()

Rows: 150300
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150300 entries, 0 to 150299
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Report_ID            150300 non-null  int64         
 1   Date                 150300 non-null  datetime64[ns]
 2   Aircraft_Make        150300 non-null  object        
 3   Aircraft_Model       150300 non-null  object        
 4   Flight_Phase         150300 non-null  object        
 5   System               150300 non-null  object        
 6   Component            149550 non-null  object        
 7   Severity             149853 non-null  object        
 8   Location             150300 non-null  object        
 9   Narrative            148697 non-null  object        
 10  Root_Cause_Keywords  150300 non-null  object        
 11  Model_Age            150300 non-null  int64         
 12  Year                 150300 non-null  int64         
 13  M

In [4]:
# Basic aggregations 

# 2. Aggregations for direct export
# 2.1 Fault counts by system, year, month
agg_system_month = df.groupby([df['Date'].dt.to_period('M').astype(str), 'System']) \
                     .size().reset_index(name='Fault_Count').rename(columns={'Date':'Month'})

# 2.2 Top components overall
top_components = df['Component'].value_counts().reset_index().rename(columns={'index':'Component', 'Component':'Fault_Count'})

# 2.3 System vs Flight Phase counts
system_phase = df.groupby(['System','Flight_Phase']).size().reset_index(name='Fault_Count')

# 2.4 System-Severity Crosstab
system_severity = pd.crosstab(df['System'], df['Severity']).reset_index()

# 2.5 Root cause by system (from your text-mining category)
root_system = df.groupby(['System','Root_Cause_Category']).size().reset_index(name='Count')

# Save CSVs for Tableau
agg_system_month.to_csv('agg_system_month.csv', index=False)
top_components.head(100).to_csv('top_components.csv', index=False)
system_phase.to_csv('system_phase.csv', index=False)
system_severity.to_csv('system_severity.csv', index=False)
root_system.to_csv('root_system.csv', index=False)

print("Exports saved: agg_system_month, top_components, system_phase, system_severity, root_system")

KeyError: 'Root_Cause_Category'